### In our last model in file "Music_Generation_Train1.ipynb", we got only 82% accuracy. However, in order to generate melodious music, we need at least 90% accuracy. 
### So, we have loaded the weights of last epoch from our previous model into this model and also we have added 2 extra layers of LSTM here with more LSTM units. 
### Here, we are fine-tuning our old layers and we have added more layers. In short, here we are doing "Transfer Learning" from old to new model.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.
/home/rcoem/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rcoem/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rcoem/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rcoem/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
data_directory = "../Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [3]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [4]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("../Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [5]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) #check documentation of train_on_batch here: https://keras.io/models/sequential/
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data2/log.csv", index = False)

In [6]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 256)             787456    
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 256)             0         
_________________________________________________________________
lstm_1 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_1 (Dropout)          (16, 64, 256)             0         
__________________________

Batch: 93, Loss: 2.9421238899230957, Accuracy: 0.1806640625
Batch: 94, Loss: 2.9310736656188965, Accuracy: 0.205078125
Batch: 95, Loss: 2.825956344604492, Accuracy: 0.2197265625
Batch: 96, Loss: 3.0161805152893066, Accuracy: 0.2001953125
Batch: 97, Loss: 2.9383087158203125, Accuracy: 0.212890625
Batch: 98, Loss: 2.974437713623047, Accuracy: 0.201171875
Batch: 99, Loss: 2.863724708557129, Accuracy: 0.203125
Batch: 100, Loss: 2.7970681190490723, Accuracy: 0.22265625
Batch: 101, Loss: 2.853343963623047, Accuracy: 0.2255859375
Batch: 102, Loss: 2.842134475708008, Accuracy: 0.2255859375
Batch: 103, Loss: 2.975924015045166, Accuracy: 0.185546875
Batch: 104, Loss: 2.844914436340332, Accuracy: 0.216796875
Batch: 105, Loss: 2.8480629920959473, Accuracy: 0.2255859375
Batch: 106, Loss: 2.8787288665771484, Accuracy: 0.21875
Batch: 107, Loss: 2.94553804397583, Accuracy: 0.2080078125
Batch: 108, Loss: 2.8997697830200195, Accuracy: 0.23046875
Batch: 109, Loss: 2.8483943939208984, Accuracy: 0.22753906

Batch: 82, Loss: 1.9669358730316162, Accuracy: 0.4169921875
Batch: 83, Loss: 1.9873906373977661, Accuracy: 0.4228515625
Batch: 84, Loss: 2.0390191078186035, Accuracy: 0.4140625
Batch: 85, Loss: 1.9981987476348877, Accuracy: 0.435546875
Batch: 86, Loss: 2.0897488594055176, Accuracy: 0.4296875
Batch: 87, Loss: 2.045060396194458, Accuracy: 0.42578125
Batch: 88, Loss: 2.118947982788086, Accuracy: 0.4072265625
Batch: 89, Loss: 2.0934371948242188, Accuracy: 0.431640625
Batch: 90, Loss: 2.0400023460388184, Accuracy: 0.4130859375
Batch: 91, Loss: 2.047128677368164, Accuracy: 0.4091796875
Batch: 92, Loss: 2.0474321842193604, Accuracy: 0.4326171875
Batch: 93, Loss: 1.9930235147476196, Accuracy: 0.4208984375
Batch: 94, Loss: 1.9862130880355835, Accuracy: 0.431640625
Batch: 95, Loss: 1.8860089778900146, Accuracy: 0.4619140625
Batch: 96, Loss: 2.0619072914123535, Accuracy: 0.431640625
Batch: 97, Loss: 1.9890440702438354, Accuracy: 0.4384765625
Batch: 98, Loss: 1.9465436935424805, Accuracy: 0.452148

Batch: 70, Loss: 1.8585331439971924, Accuracy: 0.486328125
Batch: 71, Loss: 1.7220234870910645, Accuracy: 0.4814453125
Batch: 72, Loss: 1.6928058862686157, Accuracy: 0.5068359375
Batch: 73, Loss: 1.8147783279418945, Accuracy: 0.4970703125
Batch: 74, Loss: 1.7098987102508545, Accuracy: 0.51171875
Batch: 75, Loss: 1.6281603574752808, Accuracy: 0.5107421875
Batch: 76, Loss: 1.7531743049621582, Accuracy: 0.4580078125
Batch: 77, Loss: 1.7573068141937256, Accuracy: 0.4755859375
Batch: 78, Loss: 1.7606112957000732, Accuracy: 0.5068359375
Batch: 79, Loss: 1.6284115314483643, Accuracy: 0.5546875
Batch: 80, Loss: 1.629575490951538, Accuracy: 0.5048828125
Batch: 81, Loss: 1.7257057428359985, Accuracy: 0.4541015625
Batch: 82, Loss: 1.7456574440002441, Accuracy: 0.4697265625
Batch: 83, Loss: 1.622321367263794, Accuracy: 0.53125
Batch: 84, Loss: 1.7435457706451416, Accuracy: 0.509765625
Batch: 85, Loss: 1.6073259115219116, Accuracy: 0.5380859375
Batch: 86, Loss: 1.8594424724578857, Accuracy: 0.47460

Batch: 57, Loss: 1.53426194190979, Accuracy: 0.5419921875
Batch: 58, Loss: 1.6482505798339844, Accuracy: 0.5087890625
Batch: 59, Loss: 1.4296966791152954, Accuracy: 0.5888671875
Batch: 60, Loss: 1.4468806982040405, Accuracy: 0.576171875
Batch: 61, Loss: 1.5516095161437988, Accuracy: 0.5185546875
Batch: 62, Loss: 1.557866096496582, Accuracy: 0.5224609375
Batch: 63, Loss: 1.5301374197006226, Accuracy: 0.5322265625
Batch: 64, Loss: 1.516289472579956, Accuracy: 0.5537109375
Batch: 65, Loss: 1.5527794361114502, Accuracy: 0.5380859375
Batch: 66, Loss: 1.4469985961914062, Accuracy: 0.5556640625
Batch: 67, Loss: 1.5756731033325195, Accuracy: 0.5380859375
Batch: 68, Loss: 1.647924780845642, Accuracy: 0.5283203125
Batch: 69, Loss: 1.5496736764907837, Accuracy: 0.5458984375
Batch: 70, Loss: 1.6142563819885254, Accuracy: 0.525390625
Batch: 71, Loss: 1.5366227626800537, Accuracy: 0.5224609375
Batch: 72, Loss: 1.4632935523986816, Accuracy: 0.55859375
Batch: 73, Loss: 1.5605289936065674, Accuracy: 0.

...

Batch: 115, Loss: 0.32062381505966187, Accuracy: 0.8896484375
Batch: 116, Loss: 0.30618685483932495, Accuracy: 0.91015625
Batch: 117, Loss: 0.34430861473083496, Accuracy: 0.8857421875
Batch: 118, Loss: 0.28906160593032837, Accuracy: 0.904296875
Batch: 119, Loss: 0.2799311876296997, Accuracy: 0.9052734375
Batch: 120, Loss: 0.2855115234851837, Accuracy: 0.9013671875
Batch: 121, Loss: 0.3415563106536865, Accuracy: 0.87890625
Batch: 122, Loss: 0.3126596212387085, Accuracy: 0.9013671875
Batch: 123, Loss: 0.2922990620136261, Accuracy: 0.8916015625
Batch: 124, Loss: 0.30764058232307434, Accuracy: 0.8935546875
Batch: 125, Loss: 0.3215923011302948, Accuracy: 0.896484375
Batch: 126, Loss: 0.34385228157043457, Accuracy: 0.8828125
Batch: 127, Loss: 0.27076616883277893, Accuracy: 0.91015625
Batch: 128, Loss: 0.3371348977088928, Accuracy: 0.8955078125
Batch: 129, Loss: 0.26723721623420715, Accuracy: 0.9111328125
Batch: 130, Loss: 0.35030192136764526, Accuracy: 0.8779296875
Batch: 131, Loss: 0.281395

Batch: 102, Loss: 0.33481845259666443, Accuracy: 0.8916015625
Batch: 103, Loss: 0.34086886048316956, Accuracy: 0.89453125
Batch: 104, Loss: 0.29070913791656494, Accuracy: 0.900390625
Batch: 105, Loss: 0.3356994092464447, Accuracy: 0.890625
Batch: 106, Loss: 0.26947885751724243, Accuracy: 0.916015625
Batch: 107, Loss: 0.27569299936294556, Accuracy: 0.9052734375
Batch: 108, Loss: 0.30587559938430786, Accuracy: 0.9111328125
Batch: 109, Loss: 0.30951136350631714, Accuracy: 0.892578125
Batch: 110, Loss: 0.28979387879371643, Accuracy: 0.896484375
Batch: 111, Loss: 0.2888616621494293, Accuracy: 0.9033203125
Batch: 112, Loss: 0.30938050150871277, Accuracy: 0.9013671875
Batch: 113, Loss: 0.314374178647995, Accuracy: 0.896484375
Batch: 114, Loss: 0.3335530161857605, Accuracy: 0.8876953125
Batch: 115, Loss: 0.33356913924217224, Accuracy: 0.8828125
Batch: 116, Loss: 0.3292600214481354, Accuracy: 0.884765625
Batch: 117, Loss: 0.3145586848258972, Accuracy: 0.8994140625
Batch: 118, Loss: 0.2889031767

Batch: 88, Loss: 0.32514965534210205, Accuracy: 0.8955078125
Batch: 89, Loss: 0.2588622272014618, Accuracy: 0.912109375
Batch: 90, Loss: 0.2913041114807129, Accuracy: 0.9033203125
Batch: 91, Loss: 0.3419960141181946, Accuracy: 0.884765625
Batch: 92, Loss: 0.2876131534576416, Accuracy: 0.89453125
Batch: 93, Loss: 0.29876163601875305, Accuracy: 0.896484375
Batch: 94, Loss: 0.3043653964996338, Accuracy: 0.8974609375
Batch: 95, Loss: 0.32387709617614746, Accuracy: 0.8935546875
Batch: 96, Loss: 0.34308934211730957, Accuracy: 0.88671875
Batch: 97, Loss: 0.2484024465084076, Accuracy: 0.919921875
Batch: 98, Loss: 0.3076467514038086, Accuracy: 0.8994140625
Batch: 99, Loss: 0.284262478351593, Accuracy: 0.9013671875
Batch: 100, Loss: 0.283321350812912, Accuracy: 0.9033203125
Batch: 101, Loss: 0.3120840787887573, Accuracy: 0.89453125
Batch: 102, Loss: 0.328011691570282, Accuracy: 0.8896484375
Batch: 103, Loss: 0.3403421640396118, Accuracy: 0.8916015625
Batch: 104, Loss: 0.29297783970832825, Accura

Batch: 74, Loss: 0.2539770305156708, Accuracy: 0.916015625
Batch: 75, Loss: 0.2441718578338623, Accuracy: 0.9169921875
Batch: 76, Loss: 0.30747514963150024, Accuracy: 0.8994140625
Batch: 77, Loss: 0.26519909501075745, Accuracy: 0.9111328125
Batch: 78, Loss: 0.29541391134262085, Accuracy: 0.9033203125
Batch: 79, Loss: 0.2884034812450409, Accuracy: 0.9052734375
Batch: 80, Loss: 0.27055490016937256, Accuracy: 0.8994140625
Batch: 81, Loss: 0.278829962015152, Accuracy: 0.9111328125
Batch: 82, Loss: 0.2945089340209961, Accuracy: 0.9052734375
Batch: 83, Loss: 0.2608381509780884, Accuracy: 0.9111328125
Batch: 84, Loss: 0.32693272829055786, Accuracy: 0.88671875
Batch: 85, Loss: 0.29255273938179016, Accuracy: 0.904296875
Batch: 86, Loss: 0.3163883686065674, Accuracy: 0.8984375
Batch: 87, Loss: 0.29150497913360596, Accuracy: 0.9111328125
Batch: 88, Loss: 0.3516101837158203, Accuracy: 0.8837890625
Batch: 89, Loss: 0.2889229953289032, Accuracy: 0.9091796875
Batch: 90, Loss: 0.31744417548179626, Acc

Batch: 60, Loss: 0.29408496618270874, Accuracy: 0.8994140625
Batch: 61, Loss: 0.3196299076080322, Accuracy: 0.8984375
Batch: 62, Loss: 0.25175952911376953, Accuracy: 0.91015625
Batch: 63, Loss: 0.30144160985946655, Accuracy: 0.896484375
Batch: 64, Loss: 0.2742689251899719, Accuracy: 0.91015625
Batch: 65, Loss: 0.28809884190559387, Accuracy: 0.9091796875
Batch: 66, Loss: 0.28317147493362427, Accuracy: 0.900390625
Batch: 67, Loss: 0.3003785014152527, Accuracy: 0.8935546875
Batch: 68, Loss: 0.34363648295402527, Accuracy: 0.880859375
Batch: 69, Loss: 0.3605680763721466, Accuracy: 0.8779296875
Batch: 70, Loss: 0.29854488372802734, Accuracy: 0.904296875
Batch: 71, Loss: 0.30444350838661194, Accuracy: 0.8984375
Batch: 72, Loss: 0.2988930642604828, Accuracy: 0.8984375
Batch: 73, Loss: 0.2800862193107605, Accuracy: 0.908203125
Batch: 74, Loss: 0.23516568541526794, Accuracy: 0.9296875
Batch: 75, Loss: 0.2604237198829651, Accuracy: 0.912109375
Batch: 76, Loss: 0.31104159355163574, Accuracy: 0.890

In [7]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.515805,0.299805
1,2,1.907171,0.463867
2,3,1.570911,0.540039
3,4,1.405042,0.576172
4,5,1.295637,0.600586
5,6,1.196182,0.626953
6,7,1.126311,0.654297
7,8,1.068558,0.664062
8,9,1.027782,0.685547
9,10,0.990100,0.701172
